In [363]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [364]:
###
train_per = 0.7
####
test_per = 1-train_per

#######
look_back = 5
#####
data = pd.read_csv('feedable(new)(verion-2)(lb-' + str(look_back) + '.csv')


train_Data = data.head(int(40*(train_per)*(97-look_back)))
test_Data = data.tail(int(40*(test_per)*(97-look_back)))


true_output_train = train_Data["expected_glucose"]
input_features_train = train_Data.drop(["expected_glucose","pat_no"] , axis=1)

true_output_test = test_Data["expected_glucose"]
input_features_test = test_Data.drop(["expected_glucose", "pat_no"] , axis=1)

#####
num_neigh = 5
##########


neigh = KNeighborsRegressor(n_neighbors=num_neigh)
neigh.fit(input_features_train ,true_output_train )
predict_train = neigh.predict(input_features_train)
mse_on_train = mean_squared_error(true_output_train , predict_train)
predict_test = neigh.predict(input_features_test)
mse_on_test = mean_squared_error(true_output_test , predict_test)


In [365]:
def initialiaze(train_Data , look_back , num_neigh):
    start_no = train_Data["pat_no"][0]
    end_no = train_Data["pat_no"][train_Data.shape[0]-1]
    df = (train_Data[train_Data["pat_no"]==start_no]).iloc[[0]]
    
    for i in range(start_no+1,end_no+1):
        tt = (train_Data[train_Data["pat_no"]==i]).iloc[[0]]
        df = pd.concat([df,tt])
    to_drop = []
    #df.to_csv("tempr.csv")
    for i in range(1,look_back+1):
        to_drop.append("t-" + str(i))
    to_drop.append("expected_glucose")
    to_drop.append("pat_no")
    input_features = df.drop( to_drop , axis=1)
    #input_features.to_csv("abcd.csv")
    models = []
    
    
    for i in range(1,look_back+1):
        output = df["t-"+str(i)]
        
        neigh = KNeighborsRegressor(n_neighbors=num_neigh)
        neigh.fit(input_features , output)
        models.append(neigh)
    return models
    
        

        
    
    
    
    

    
        


In [366]:
a = initialiaze(train_Data , look_back , num_neigh)

In [367]:
def ext(test_list , insert_list , pos):
    for i in range(len(insert_list)): 
        test_list.insert(i + pos, insert_list[i][0]) 

def sequence(const_para , models , main_model):
    seq = []
    for i in range(look_back):
        seq.append(models[i].predict(const_para))
    
    pos = 24
    for i in range(look_back+1 , 97+1):
        history = seq[-look_back:]
        tempp = list(const_para[:].values)
        tempp = (list(tempp[0]))
        ext(tempp , history , pos+1)
        #print(tempp)
        seq.append((main_model.predict([tempp])))
    #print(tempp)
    return seq
        
        
    

In [368]:
import numpy as np
for j in range(29,30):
    tt = (test_Data[test_Data["pat_no"]==j]).iloc[[0]]
    to_drop = []
    for i in range(1,look_back+1):
        to_drop.append("t-" + str(i))
    to_drop.append("expected_glucose")
    to_drop.append("pat_no")
    tt = tt.drop( to_drop , axis=1)
    pos = list(tt).index("hi4")
    qq = sequence(tt , a , neigh)
    np.savetxt(str(j)+".csv", qq, delimiter=",")